In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=1f0621ca702919bb957a6dc132244232b81f90a2f28764e6310402af2205b457
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from operator import add

# Initialize a Spark session
spark = SparkSession.builder.appName("PageRank").getOrCreate()
sc = SparkContext.getOrCreate()
edges_rdd = sc.textFile("/content/pagerank.txt").map(lambda line: tuple(line.split(',')))

ranks = edges_rdd.map(lambda edge: (edge[0], 1.0))


In [5]:
for _ in range(20):
    # Calculate contributions from each page
    contributions = edges_rdd.join(ranks).flatMap(
        lambda x: [(x[1][0], x[1][1] / len(x[1][0]))]
    )

    # Sum up the contributions for each page
    ranks = contributions.reduceByKey(add).mapValues(lambda rank: 0.15 + 0.85 * rank)


In [6]:
final_ranks = ranks.collect()
print("Final PageRank scores:")
for page, rank in final_ranks:
    print(f"{page}: {rank}")

Final PageRank scores:
C: 222.38234724411487
D: 359.73128200360384
F: 266.82844522299706
A: 165.01410259049533
E: 266.82844522299706
G: 207.5789167524176
B: 335.86633446021517
H: 207.5789167524176
